# Comparing the reconstruction results of different methods

In [ ]:
name1 = "moero-mini-fine"
name2 = "moero-0.04"
name3 = "zf"
name4 = "sensegrappa"

patha = f"/home/hulabdl/exp/taskr1submit/{name1}"
pathb = f"/home/hulabdl/exp/taskr1submit/{name2}"

pathc = f"/home/hulabdl/exp/taskr1submit/{name3}"
pathd = f"/home/hulabdl/exp/taskr1submit/{name4}"

save_fig_to = f".imgs/moero-mini-debug"

from glob import glob
from os.path import join
import os
import scipy
import random
from matplotlib import pyplot as plt
import scipy.io
import numpy as np

os.makedirs(save_fig_to, exist_ok=True)

fnamelista = sorted(glob(join(patha, "**/*.mat"), recursive=True))
print(len(fnamelista))

plt.figure(figsize=(20, 10))

for fnamea in fnamelista:
    fnameb = fnamea.replace(patha, pathb)
    fnamec = fnamea.replace(patha, pathc)
    fnamed = fnamea.replace(patha, pathd)

    dataa = scipy.io.loadmat(fnamea)["img4ranking"]
    try:
        datab = scipy.io.loadmat(fnameb)["img4ranking"]
        datac = scipy.io.loadmat(fnamec)["img4ranking"]
        datad = scipy.io.loadmat(fnamed)["img4ranking"]
    except Exception as e:
        continue

    if len(dataa.shape) == 4:
        iframe = random.choice(list(range(dataa.shape[-1])))
        islice = random.choice(list(range(dataa.shape[-2])))
        dataa = dataa[:, :, islice, iframe]
        datab = datab[:, :, islice, iframe]
        datac = datac[:, :, islice, iframe]
        datad = datad[:, :, islice, iframe]
    else:
        islice = random.choice(list(range(dataa.shape[-1])))
        dataa = dataa[:, :, islice]
        datab = datab[:, :, islice]
        datac = datac[:, :, islice]
        datad = datad[:, :, islice]

    plt.subplot(1, 4, 1)
    plt.imshow(dataa, cmap="gray")
    plt.title(name1)

    plt.subplot(1, 4, 2)
    plt.imshow(datab, cmap="gray")
    plt.title("moero")

    plt.subplot(1, 4, 3)
    plt.imshow(datac, cmap="gray")
    plt.title(name3)

    plt.subplot(1, 4, 4)
    plt.imshow(datad, cmap="gray")
    plt.title(name4)

    # plt.colorbar()

    # Save the figure and close it to avoid displaying it in the notebook output
    save_path = join(save_fig_to, "@".join(fnamea.split("/")[-7:]).replace(".mat", ".png"))
    plt.savefig(save_path)
    plt.close()  # Close the figure to avoid displaying it in the notebook output

In [ ]:
path = "/home/hulabdl/CMRxRecon2025/TaskR1/MultiCoil/Mapping/ValidationSet/UnderSample_TaskR1/Center005/GE_15T_voyager/P004/T1map_kus_ktGaussian8.mat"

import h5py
import numpy as np
from matplotlib import pyplot as plt

def rsos(img):
    img = np.abs(img)
    img = img * img
    img = np.sum(img, axis=0)
    img = np.sqrt(img)
    return img

with h5py.File(path, "r") as f:
    data = f["kus"][:]
    data = data['real'] + 1j * data['imag']
    data = data[7, 1]
    print(data.shape)
    plt.figure()
    plt.imshow(np.log(np.abs(data[0]) + 1e-13), cmap="gray")
    plt.show()

    img = np.fft.ifftn(data, axes=(1, 2))
    img = np.fft.fftshift(img, axes=(1, 2))
    img = rsos(img)
    img = np.abs(img)
    print(np.linalg.norm(img), np.mean(img))
    plt.figure()
    plt.imshow(img, cmap="gray")
    plt.colorbar()
    plt.show()


In [ ]:
import torch
import math
from ..utils.naneu.fft import ktoi

path = "/home/hulabdl/exp/taskr1out/novq-single/Mapping@ValidationSet@UnderSample_TaskR1@Center005@GE_15T_voyager@P004@T1map_kus_ktGaussian8.pt.zf"

data = torch.load(path)
print(data.shape)

img = ktoi(data[1,1,0])
# img = img.sum(0)
img = img.swapaxes(-1,-2)

# crop center 50
# w = 20
# img = img[math.floor(img.shape[0]/2) + math.ceil(-w/2):math.floor(img.shape[0]/2) + math.floor(w/2),
#           math.floor(img.shape[1]/2) + math.ceil(-w/2):math.floor(img.shape[1]/2) + math.floor(w/2)]
print(img.angle().max())
# print(img.max(), img.min(), img.mean(), img.std()) 
plt.figure()
plt.imshow(img.abs(), cmap = "gray")
plt.colorbar()
plt.show()
